## Storage and Retrieval
*If you keep things tidily ordered, you’re just too lazy to go searching. —German proverb*

Storage engines being used in tradtional and NoSql Databases


### Data Structures that power your Database


- In order to efficiently find the value for a particular key in the database, we need a different data structure: an index.
- we will look at a range of indexing structures and see how they compare; the general idea behind them is to keep some additional metadata on the side, which acts as a signpost and helps you to locate the data you want.
- If you want to search the same data in several different ways, you may need several different indexes on different parts of the data.
- *An index is an additional structure that is derived from the primary data. Many databases allow you to add and remove indexes, and this doesn’t affect the contents of the database; it only affects the performance of queries.*

- This is an important trade-off in storage systems: well-chosen indexes speed up read queries, but every index slows down writes. For this reason, databases don’t usually index everything by default, but require you—the application developer or database administrator—to choose indexes manually, using your knowledge of the applica‐ tion’s typical query patterns.
  

##### Hash Indexes

*Compaction means throwing away duplicate keys in the log, and keeping only the most recent update for each key.*

Key-value data indexing is a fundamental approach used in databases, similar to hash maps in programming languages. This method involves maintaining an in-memory hash map that maps keys to byte offsets in a data file, allowing quick value retrieval by seeking to the specified location. This strategy, used by systems like Bitcask in Riak, provides high-performance reads and writes as long as all keys fit in memory, though values can exceed available RAM. To prevent running out of disk space, data is written in segments and compacted to remove duplicate keys, retaining only the latest value for each key. During compaction, segments can be merged in the background, allowing continuous read and write operations. Each segment maintains its own hash table, and lookups proceed through these tables starting from the most recent segment to minimize search time.


* File Format: CSV is not the best format for a log, its faster and simpler to use a binary format that first encodes the length of a string in bytes, followed by the raw string. (without need for escaping)
* Deleting the records: if you want to delete a key and its associated value, you have to append a special deletion record, to data file(sometimes calledas tombstone). whne log segments are merged, the tombstone tells the merging process to discard any previous values for the deleted key.
* Crash recovery: If the database is restarted, the in-memory hash maps are lost. In principle, you can restore each segment’s hash map by reading the entire segment file from beginning to end and noting the offset of the most recent value for every key as you go along. However, that might take a long time if the segment files are large, which would make server restarts painful. Bitcask speeds up recovery by storing a snapshot of each segment’s hash map on disk, which can be loaded into memory more quickly.
* Partially written records: The database may crash at any time, including halfway through appending a record to the log. Bitcask files include checksums, allowing such corrupted parts of the log to be detected and ignored.
* Concurrency control: As writes are appended to the log in a strictly sequential order, a common imple‐ mentation choice is to have only one writer thread. Data file segments are append-only and otherwise immutable, so they can be read concurrently by multiple threads.



An append-only design turns out to be good for several reasons: 
* Appending and segment merging are sequential write operations, which are gen‐ erally much faster than random writes, especially on magnetic spinning-disk hard drives. To some extent sequential writes are also preferable on flash-based solid state drives (SSDs)
* Concurrency and crash recovery are much simpler if segment files are append- only or immutable. For example, you don’t have to worry about the case where a crash happened while a value was being overwritten, leaving you with a file con‐ taining part of the old and part of the new value spliced together.
* Merging old segments avoids the problem of data files getting fragmented over time.
